In [1]:
import cv2 
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Model
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, Conv2D
)
from keras.layers import LeakyReLU

img = cv2.imread('C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img/Inundaciones/frame_2.jpg')
print(img.shape[0], img.shape[1], img.shape[2], len(img.shape))

#dirname = os.path.join(os.getcwd(),'C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/View/img/i1.jpeg')
dirname = os.path.join(os.getcwd(),'C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            if(len(image.shape)==3):
                images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1
y = np.array(labels)
X = np.array(images, dtype=np.uint8)

train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X/255.
test_X = test_X/255.

train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

nClasses = len(deportes)

INIT_LR = 1e-3
epochs = 20
batch_size = 64
sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))
sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5))
# Utilizamos nClasses en la capa Dense
sport_model.add(Dense(nClasses, activation='softmax'))
sport_model.summary()

# Compilación del modelo
INIT_LR = 1e-3
optimizer = tf.keras.optimizers.SGD(learning_rate=INIT_LR, decay=INIT_LR / 100)
sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

sport_train = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
sport_model.save("C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/ccn.h5")

test_eval = sport_model.evaluate(test_X, test_Y_one_hot, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

accuracy = sport_train.history['accuracy']
val_accuracy = sport_train.history['val_accuracy']
loss = sport_train.history['loss']
val_loss = sport_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

predicted_classes2 = sport_model.predict(test_X)
predicted_classes=[]
for predicted_sport in predicted_classes2:
    predicted_classes.append(predicted_sport.tolist().index(max(predicted_sport)))
predicted_classes=np.array(predicted_classes)
correct = np.where(predicted_classes==test_Y)[0]
incorrect = np.where(predicted_classes!=test_Y)[0]

from skimage.transform import resize
images=[]
filenames = ['C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img/Tornados/frame_6867.jpg']

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (21, 28),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8)
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = sport_model.predict(test_X)

#for i, img_tagged in enumerate(predicted_classes):
    #print(filenames[i], deportes[img_tagged.tolist().index(max(img_tagged))])
    
   # predicted_classes=np.array(predicted_classes)

###########
predicted_classes=[]
for predicted_sport in predicted_classes2:
    predicted_classes.append(predicted_sport.tolist().index(max(predicted_sport)))
predicted_classes=np.array(predicted_classes)

correct = np.where(predicted_classes == test_Y)[0]
if len(correct) > 0:
    print("Found %d correct labels" % len(correct))
    for i, correct_index in enumerate(correct[:min(9, len(correct))]):
        plt.subplot(3, 3, i + 1)
        plt.imshow(test_X[correct_index].reshape(21, 28, 3), cmap='gray', interpolation='none')
        plt.title("{}, {}".format(deportes[predicted_classes[correct_index]], deportes[test_Y[correct_index]]))
        plt.tight_layout()     
else:
    print("No correct labels found.")

incorrect = np.where(predicted_classes != test_Y)[0]
if len(incorrect) > 0:
    print("Found %d incorrect labels" % len(incorrect))
    for i, incorrect_index in enumerate(incorrect[:min(9, len(incorrect))]):  # Limitamos a 9 elementos o la longitud de incorrect si es menor
        plt.subplot(3, 3, i + 1)
        plt.imshow(test_X[incorrect_index].reshape(21, 28, 3), cmap='gray', interpolation='none')
        plt.title("{}, {}".format(deportes[predicted_classes[incorrect_index]], deportes[test_Y[incorrect_index]]))
        plt.tight_layout()    
else:
    print("No incorrect labels found.")

target_names = ["Class {}".format(i) for i in range(nClasses)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))



<>:37: SyntaxWarning: invalid escape sequence '\.'
<>:37: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Antonio\AppData\Local\Temp\ipykernel_3360\2302927227.py:37: SyntaxWarning: invalid escape sequence '\.'
  if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):


21 28 3 3
leyendo imagenes de  C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img\
C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img\Incendios 1
C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img\Inundaciones 4568
C:/Projects/ITM/INTELIGENCIA ARTIFICIAL/4_Proyecto_CNN/Img\Tornados 1364
Directorios leidos: 3
Imagenes en cada directorio [4569, 1364, 1580]
suma Total de imagenes en subdirs: 7513
Cantidad etiquetas creadas:  7513
0 Incendios
1 Inundaciones
2 Tornados


C:\Users\Antonio\AppData\Local\Temp\ipykernel_3360\2302927227.py:37: SyntaxWarning: invalid escape sequence '\.'
  if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (7513,) + inhomogeneous part.